In [1]:
import modelseedpy
import cobrakbase

cobrakbase 0.2.8


In [2]:
modelseed = modelseedpy.biochem.from_local('../../../ModelSEEDDatabase')

In [4]:
for r in modelseed.reactions:
    print(r.delta_g, r.delta_g_error, type(r.delta_g))

-16.82 1.56 <class 'float'>
-114.63 2.23 <class 'float'>
10000000.0 10000000.0 <class 'float'>
0.53 0.54 <class 'float'>
-2.6 0.56 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
-0.68 6.88 <class 'float'>
-105.97 0.91 <class 'float'>
6.11 4.35 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
10000000.0 10000000.0 <class 'float'>
-0.74 0.51 <class 'float'>
-11.84 0.42 <class 'float'>
-48.9 5.79 <class 'float'>
-13.17 0.99 <class 'float'>
-0.74 0.51 <class 'float'>
-0.74 0.51 <class 'float'>
-53.3 0.85 <class 'float'>
-6.09 7.63 <class 'float'>
75.94 7.57 <class 'fl

KeyboardInterrupt: 

In [17]:
import os
import json
from modelseedpy.biochem.modelseed_compound import ModelSEEDCompound2
database_path = '/home/fliu/workspace/python3/ModelSEEDDatabase'
_BIOCHEM_FOLDER = 'Biochemistry'
def load_metabolites(database_path):
    metabolites = {}
    contents = os.listdir(f'{database_path}/{_BIOCHEM_FOLDER}')
    for f in contents:
        if f.startswith('compound_') and f.endswith('.json'):
            with open(f'{database_path}/{_BIOCHEM_FOLDER}/{f}', 'r') as fh:
                _compounds_data = json.load(fh)
                for o in _compounds_data:
                    if 'id' in o and o['id']:
                        cpd = ModelSEEDCompound2(o['id'], o.get('formula'), 
                                                 o.get('name'), o.get('charge'), '',
                                                 o.get('abbreviation'), None,
                                                 o.get('mass'), o.get('deltag'), o.get('deltagerr'))
                        metabolites[cpd.id] = cpd
                    else:
                        print('error', o)
                #print(_compounds_data[0].keys())
        
    return metabolites
metabolites = load_metabolites(database_path)

In [18]:
from modelseedpy.biochem.modelseed_reaction import ModelSEEDReaction2
from modelseedpy.core.msmodel import get_reaction_constraints_from_direction
def load_reactions(database_path, metabolites):
    reactions = {}
    contents = os.listdir(f'{database_path}/{_BIOCHEM_FOLDER}')
    metabolites_indexed = {}
    for f in contents:
        if f.startswith('reaction_') and f.endswith('.json'):
            with open(f'{database_path}/{_BIOCHEM_FOLDER}/{f}', 'r') as fh:
                _reactions_data = json.load(fh)
                for o in _reactions_data:
                    if 'id' in o and o['id']:
                        lower_bound, upper_bound = get_reaction_constraints_from_direction(o.get('reversibility'))
                        stoichiometry = o.get('stoichiometry')
                        reaction_metabolites = {}
                        for s in stoichiometry:
                            cmp_token = s['compartment']
                            value = s['coefficient']
                            cpd = metabolites[s['compound']]
                            cpd_index_id = f"{cpd.id}_{cmp_token}"
                            if cpd_index_id not in metabolites_indexed:
                                cpd_token = cpd.copy()
                                cpd_token.id = f"{cpd.id}_{cmp_token}"
                                cpd_token.base_id = cpd.id
                                cpd_token.compartment = cmp_token
                                metabolites_indexed[cpd_index_id] = cpd_token
                            reaction_metabolites[metabolites_indexed[cpd_index_id]] = value
                        rxn = ModelSEEDReaction2(o['id'], o.get('name'), '', lower_bound, upper_bound, delta_g=_reactions_data.get('deltag'), delta_g_error=_reactions_data.get(deltagerr))
                        rxn.add_metabolites(reaction_metabolites)
                        reactions[rxn.id] = rxn
                    else:
                        print('error', o)
                print(_reactions_data[0])
            break
    return reactions
reactions = load_reactions(database_path, metabolites)

{'abbreviation': 'RXN-11623.v', 'abstract_reaction': None, 'aliases': ['AraCyc: RXN-11623', 'CornCyc: RXN-11623', 'KEGG: R09562', 'MetaCyc: RXN-11623', 'PlantCyc: RXN-11623', 'Name: FC lyase; FCLY; S-(2E,6E)-farnesyl-L-cysteine oxidase; farnesylcysteine lyase'], 'code': '(1) cpd00001[0] + (1) cpd00007[0] + (1) cpd20939[0] <=> (1) cpd00025[0] + (1) cpd00084[0] + (1) cpd02188[0]', 'compound_ids': 'cpd00001;cpd00007;cpd00025;cpd00084;cpd02188;cpd20939', 'definition': '(1) H2O[0] + (1) O2[0] + (1) Farnesylcysteine[0] => (1) H2O2[0] + (1) L-Cysteine[0] + (1) 2-trans,6-trans-Farnesal[0]', 'deltag': -16.82, 'deltagerr': 1.56, 'ec_numbers': ['1.8.3.5', '1.8.3.6'], 'equation': '(1) cpd00001[0] + (1) cpd00007[0] + (1) cpd20939[0] => (1) cpd00025[0] + (1) cpd00084[0] + (1) cpd02188[0]', 'id': 'rxn22000', 'is_obsolete': 1, 'is_transport': 0, 'linked_reaction': 'rxn16406', 'name': 'farnesylcysteine lyase', 'notes': ['GCC', 'EQC', 'EQU'], 'pathways': ['MetaCyc: All-Trans-Farnesyl-PP-Biosynthesis ();

In [3]:
type(modelseed)

modelseedpy.biochem.modelseed_biochem.ModelSEEDBiochem

In [5]:
modelseed.compound_aliases['cpd00001']['KEGG']

{'C00001', 'C01328'}

In [2]:
kbase = cobrakbase.KBaseAPI()

In [102]:
template = kbase.get_from_ws('CoreBacteria_updated', 12218)
print(template.info)

12218/444/1


In [103]:
from cobrakbase.core.kbasefba.newmodeltemplate_metabolite import NewModelTemplateCompound
compounds = {}
for cc in template.compcompounds:
    if cc.compound is None:
        cpd = modelseed.get_seed_compound(cc.id[:-2])
        if cpd.id not in compounds:
            template_compound = NewModelTemplateCompound(cpd.id, cpd.formula, cpd.name)
            compounds[template_compound.id] = NewModelTemplateCompound(cpd.id, cpd.formula, cpd.name)
            print(cpd)

In [101]:
kbase.save_object('CoreBacteria_updated', 12218, template.info.type, template)

In [100]:
template.add_compounds(list(compounds.values()))

In [91]:

template_compound

Compound identifier,cpd26984
Name,DsrC-disulfide-form
Memory address,0x07fc27bc0b710
Formula,C6H9N2O2R3S2
In 0 species,


Compound identifier,cpd26984
Name,DsrC-disulfide-form
Memory address,0x07fc27bb9ea50
Formula,C6H9N2O2R3S2
In 0 species,


In [104]:
template_reaction = template.reactions.rxa45615_c

In [17]:
template_reaction.get_data()

{'id': 'rxa45615_c',
 'name': 'rxa45615_c',
 'GapfillDirection': '=',
 'base_cost': 1000,
 'reverse_penalty': 1000,
 'forward_penalty': 1000,
 'upper_bound': 1000,
 'lower_bound': -1000,
 'direction': '=',
 'maxforflux': 1000,
 'maxrevflux': 1000.0,
 'reaction_ref': 'kbase/default/reactions/id/rxa45615',
 'templateReactionReagents': [{'coefficient': -2,
   'templatecompcompound_ref': '~/compcompounds/id/cpd00067_c'},
  {'coefficient': -3,
   'templatecompcompound_ref': '~/compcompounds/id/cpd00971_c'},
  {'coefficient': -2,
   'templatecompcompound_ref': '~/compcompounds/id/cpd11620_c'},
  {'coefficient': -1,
   'templatecompcompound_ref': '~/compcompounds/id/cpd08701_c'},
  {'coefficient': 2,
   'templatecompcompound_ref': '~/compcompounds/id/cpd00067_e'},
  {'coefficient': 3,
   'templatecompcompound_ref': '~/compcompounds/id/cpd00971_e'},
  {'coefficient': 2,
   'templatecompcompound_ref': '~/compcompounds/id/cpd11621_c'},
  {'coefficient': 1,
   'templatecompcompound_ref': '~/compc

In [105]:
for o in template_reaction.metabolites:
    print(type(o), id(o), o.id, o.compound, o.name)

<class 'cobrakbase.core.kbasefba.newmodeltemplate_metabolite.NewModelTemplateCompCompound'> 140472454487120 cpd00067_c cpd00067:H+ H+
<class 'cobrakbase.core.kbasefba.newmodeltemplate_metabolite.NewModelTemplateCompCompound'> 140472433862480 cpd00971_c cpd00971:Na+ Na+
<class 'cobrakbase.core.kbasefba.newmodeltemplate_metabolite.NewModelTemplateCompCompound'> 140472454486480 cpd11620_c cpd11620:Reducedferredoxin Reducedferredoxin
<class 'cobrakbase.core.kbasefba.newmodeltemplate_metabolite.NewModelTemplateCompCompound'> 140472433931728 cpd08701_c cpd08701:Methanophenazine Methanophenazine
<class 'cobrakbase.core.kbasefba.newmodeltemplate_metabolite.NewModelTemplateCompCompound'> 140472433821840 cpd00067_e cpd00067:H+ H+
<class 'cobrakbase.core.kbasefba.newmodeltemplate_metabolite.NewModelTemplateCompCompound'> 140472433861840 cpd00971_e cpd00971:Na+ Na+
<class 'cobrakbase.core.kbasefba.newmodeltemplate_metabolite.NewModelTemplateCompCompound'> 140472433893520 cpd11621_c cpd11621:Oxidiz

In [65]:
template.compcompounds.cpd08701_c.get_data()

{'charge': 0,
 'id': 'cpd08701_c',
 'maxuptake': 0,
 'templatecompartment_ref': '~/compartments/id/c',
 'templatecompound_ref': '~/compounds/id/cpd08701'}

In [58]:
id(template.compcompounds.cpd08701_c.cpd08701_c)

140472724747984

In [61]:
template.compcompounds.cpd08701_c

Metabolite identifier,cpd08701_c
Name,
Memory address,0x07fc25936bb90
Formula,
Compartment,c
In 3 reaction(s),"rxn03126_c, rxa45615_c, rxn15961_c"


In [134]:
i =0 
for r in modelseed.reactions:
    print(modelseed.reactions[r]['code'], modelseed.reactions[r]['direction'], modelseed.reactions[r]['reversibility'])
    #print(modelseed.reactions[r]['code'])
    #print(modelseed.reactions[r]['stoichiometry'])
    #print(modelseed.reactions[r]['definition'])
    
    
    i+= 1
    if i > 10:
        break

(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd00009[0] = >
(1) cpd00001[0] + (1) cpd00742[0] <=> (2) cpd00011[0] + (2) cpd00013[0] > >
(1) cpd00011[0] + (1) cpd00668[0] <=> (2) cpd00020[0] < <
(1) cpd02570[0] <=> (2) cpd00020[0] = =
(2) cpd00025[0] <=> (2) cpd00001[0] + (1) cpd00007[0] > >
(1) cpd00001[0] + (1) cpd00794[0] <=> (2) cpd00027[0] > =
(2) cpd00001[0] <=> (1) cpd00025[0] = <
(2) cpd00038[0] <=> (1) cpd00012[0] + (1) cpd00925[0] > =
(2) cpd00040[0] <=> (1) cpd00011[0] + (1) cpd00843[0] > =
(1) cpd00011[0] + (1) cpd03049[0] <=> (1) cpd00020[0] + (1) cpd00056[0] > <
(2) cpd00076[0] <=> (1) cpd00027[0] + (1) cpd02298[0] = =


In [153]:
import math
from modelseedpy.core.msmodel import get_direction_from_constraints
eq_symbol = {'=': '<=>', '>': '=>', '<': '<='}
def to_biochem(template_reaction, compartments: dict, source=None):
    direaction_symbol = get_direction_from_constraints(template_reaction.lower_bound, template_reaction.upper_bound)
    data = {
        'id': template_reaction.reference_id,
        'abbreviation': '',
        'name': template_reaction.name,
        'code': '(1) cpd00003[0] + (1) cpd00971[0] + (2) cpd28082[0] <=> (1) cpd00004[0] + (1) cpd00971[1] + (2) cpd27757[0]',
        'stoichiometry': '-1:cpd00003:0:0:NAD;-1:cpd00067:0:0:H+;-1:cpd00971:0:0:Na+;-2:cpd28082:0:0:Reduced-ferredoxins;1:cpd00004:0:0:NADH;1:cpd00971:1:0:Na+;2:cpd27757:0:0:Oxidized-ferredoxins',
        'is_transport': 0,
        'equation': '(1) cpd00003[0] + (1) cpd00067[0] + (1) cpd00971[0] + (2) cpd28082[0] <=> (1) cpd00004[0] + (1) cpd00971[1] + (2) cpd27757[0]',
        'definition': '(1) NAD[0] + (1) H+[0] + (1) Na+[0] + (2) Reduced-ferredoxins[0] <=> (1) NADH[0] + (1) Na+[1] + (2) Oxidized-ferredoxins[0]',
        'reversibility': direaction_symbol,
        'direction': direaction_symbol,
        'abstract_reaction': '',
        'pathways': '',
        'aliases': '',
        'ec_numbers': '',
        'deltag': 10000000.0,
        'deltagerr': 10000000.0,
        'compound_ids': 'cpd00003;cpd00004;cpd00067;cpd00971;cpd27757;cpd28082',
        'status': 'OK',
        'is_obsolete': 0,
        'linked_reaction': 'nan',
        'notes': 'GCP|EQP',
        'source': source
    }
    lhs = list(filter(lambda x: x[1] < 0, template_reaction.metabolites.items()))
    rhs = list(filter(lambda x: x[1] > 0, template_reaction.metabolites.items()))
    map_code_lhs = list(map(lambda x: "({}) {}[{}]".format(math.fabs(x[1]), x[0].compound.id, compartments[x[0].compartment]), lhs))
    map_code_rhs = list(map(lambda x: "({}) {}[{}]".format(math.fabs(x[1]), x[0].compound.id, compartments[x[0].compartment]), rhs))
    map_name_lhs = map(lambda x: "({}) {}[{}]".format(math.fabs(x[1]), x[0].compound.name, compartments[x[0].compartment]), lhs)
    map_name_rhs = map(lambda x: "({}) {}[{}]".format(math.fabs(x[1]), x[0].compound.name, compartments[x[0].compartment]), rhs)
    map_stoich = map(lambda x: "{}:{}:{}:{}:{}".format(x[1], x[0].compound.id, compartments[x[0].compartment], 0, x[0].compound.name), template_reaction.metabolites.items())
    data['stoichiometry'] = ';'.join(map_stoich)
    data['code'] = "{} <=> {}".format(' + '.join(map_code_lhs), ' + '.join(map_code_rhs)).strip()
    data['equation'] = "{} {} {}".format(' + '.join(map_code_lhs), eq_symbol[direaction_symbol], ' + '.join(map_code_rhs)).strip()
    data['definition'] = "{} {} {}".format(' + '.join(map_name_lhs), eq_symbol[direaction_symbol], ' + '.join(map_name_rhs)).strip()
    return data
to_biochem(template.reactions.rxa45615_c, {'c': 0, 'e': 1})

{'id': 'rxa45615',
 'abbreviation': '',
 'name': 'rxa45615_c',
 'code': '(2.0) cpd00067[0] + (3.0) cpd00971[0] + (2.0) cpd11620[0] + (1.0) cpd08701[0] <=> (2.0) cpd00067[1] + (3.0) cpd00971[1] + (2.0) cpd11621[0] + (1.0) cpd08702[0]',
 'stoichiometry': '-2:cpd00067:0:0:H+;-3:cpd00971:0:0:Na+;-2:cpd11620:0:0:Reducedferredoxin;-1:cpd08701:0:0:Methanophenazine;2:cpd00067:1:0:H+;3:cpd00971:1:0:Na+;2:cpd11621:0:0:Oxidizedferredoxin;1:cpd08702:0:0:Dihydromethanophenazine',
 'is_transport': 0,
 'equation': '(2.0) cpd00067[0] + (3.0) cpd00971[0] + (2.0) cpd11620[0] + (1.0) cpd08701[0] <=> (2.0) cpd00067[1] + (3.0) cpd00971[1] + (2.0) cpd11621[0] + (1.0) cpd08702[0]',
 'definition': '(2.0) H+[0] + (3.0) Na+[0] + (2.0) Reducedferredoxin[0] + (1.0) Methanophenazine[0] <=> (2.0) H+[1] + (3.0) Na+[1] + (2.0) Oxidizedferredoxin[0] + (1.0) Dihydromethanophenazine[0]',
 'reversibility': '=',
 'direction': '=',
 'abstract_reaction': '',
 'pathways': '',
 'aliases': '',
 'ec_numbers': '',
 'deltag': 100

In [157]:
extra_reactions = {}
extract_ids = ['sul00002_c', 'sul00003_c', 'sul00004_c', 'sul00008_c', 'sul00010_c', 'rxa45615_c']
for k in extract_ids:
    res = to_biochem(template.reactions.get_by_id(k), {'c': 0, 'e': 1}, 'jose')
    extra_reactions[res['id']] = res

In [159]:
import json
with open('/Users/fliu/workspace/jupyter/python3/annotation-server/data/extra_reactions.json', 'w') as fh:
    fh.write(json.dumps(extra_reactions))


In [ ]:
modelseed.reactions.update

In [156]:
rxn = modelseed.get_seed_reaction('rxn45615')
print(type(rxn))
print(rxn)

<class 'modelseedpy.biochem.modelseed_reaction.ModelSEEDReaction'>
rxn45615: cpd00003 [0] + cpd00067 [0] + cpd00971 [0] + 2.0 cpd28082 [0] <=> cpd00004 [0] + cpd00971 [1] + 2.0 cpd27757 [0]


In [10]:
modelseed.reactions['rxn45615']

{'id': 'rxn45615',
 'abbreviation': nan,
 'name': nan,
 'code': '(1) cpd00003[0] + (1) cpd00971[0] + (2) cpd28082[0] <=> (1) cpd00004[0] + (1) cpd00971[1] + (2) cpd27757[0]',
 'stoichiometry': '-1:cpd00003:0:0:NAD;-1:cpd00067:0:0:H+;-1:cpd00971:0:0:Na+;-2:cpd28082:0:0:Reduced-ferredoxins;1:cpd00004:0:0:NADH;1:cpd00971:1:0:Na+;2:cpd27757:0:0:Oxidized-ferredoxins',
 'is_transport': 1,
 'equation': '(1) cpd00003[0] + (1) cpd00067[0] + (1) cpd00971[0] + (2) cpd28082[0] <=> (1) cpd00004[0] + (1) cpd00971[1] + (2) cpd27757[0]',
 'definition': '(1) NAD[0] + (1) H+[0] + (1) Na+[0] + (2) Reduced-ferredoxins[0] <=> (1) NADH[0] + (1) Na+[1] + (2) Oxidized-ferredoxins[0]',
 'reversibility': '?',
 'direction': '=',
 'abstract_reaction': nan,
 'pathways': nan,
 'aliases': 'MetaCyc: TRANS-RXN-276',
 'ec_numbers': '7.2.1.2',
 'deltag': 10000000.0,
 'deltagerr': 10000000.0,
 'compound_ids': 'cpd00003;cpd00004;cpd00067;cpd00971;cpd27757;cpd28082',
 'status': 'OK',
 'is_obsolete': 0,
 'linked_reaction': 